In [1]:
from data_utils import Data_Utilities
from dataset import DebugBinaryClassificationDatasetThreePaths, BinaryClassificationDatasetVal
from transforms import Transforms


import os, time, warnings
import numpy as np

# Dependencies for network, loss, etc.
import torch, torchvision
import torchvision.transforms.functional as functional
import eval_utils, losses
from networks import RadarNet
from typing import NamedTuple

In [2]:
# Dependencies for logging
import log_utils
from log_utils import log
from torch.utils.tensorboard import SummaryWriter

import pickle

warnings.filterwarnings("ignore")


class Data_Struct(NamedTuple):
    scene_id: int
    sample_idx: int
    image_path: str
    ground_truth_points: np.ndarray
    input_points: np.ndarray
    ground_truth_label_path: str
    ground_truth_depth_path: str# Dependencies for logging
import log_utils
from log_utils import log
from torch.utils.tensorboard import SummaryWriter

import pickle

warnings.filterwarnings("ignore")


class Data_Struct(NamedTuple):
    scene_id: int
    sample_idx: int
    image_path: str
    ground_truth_points: np.ndarray
    input_points: np.ndarray
    ground_truth_label_path: str
    ground_truth_depth_path: str

In [3]:
# path to the file that contains paths for pseudo GT training set
open_file = open('../pseudo_gt_label_paths.pkl', "rb")
pseudo_gt_train_paths = pickle.load(open_file)
open_file.close()

# path to gt_paths
open_file = open('../gt_images_paths.pkl', "rb")
gt_paths = pickle.load(open_file)
open_file.close()

# path to radar numpy arrays
open_file = open('../radar_numpy_paths.pkl', "rb")
radar_paths = pickle.load(open_file)
open_file.close()


image_path = '/home/akash/Documents/nuScenes/samples/CAM_FRONT'

data_path = '../'


data_utilties_object_val = Data_Utilities('../val_nuScenes_dataset_lidar_maps_interpolated_merged_6_6_100_with_filter.pkl')


patch_size = [768, 288]
batch_size = 64

num_workers = 1

normalized_image_range = [0,1]
augmentation_random_brightness = [-1, -1]
augmentation_random_noise_spread = 0.0

val_radar_input_points, \
    val_image_path, \
    val_scene_id_final, \
    val_sample_idx_final, \
    val_shift = data_utilties_object_val.load_data_val()


debug = False

if debug:
    end_train_list = 100
    end_val_list = end_train_list + 50

    train_dataset = DebugBinaryClassificationDatasetThreePaths(
        pseudo_gt_paths=pseudo_gt_train_paths[0:end_train_list],
        gt_paths=gt_paths[0:end_train_list],
        radar_paths=radar_paths[0:end_train_list],
        image_path=image_path,
        data_path=data_path,
        patch_size=patch_size)
    val_dataset = BinaryClassificationDatasetVal(
        image_list=val_image_path[end_train_list:end_val_list],
        radar_points=val_radar_input_points[end_train_list:end_val_list],
        patch_size=patch_size)
else:
    train_dataset = DebugBinaryClassificationDatasetThreePaths(
        pseudo_gt_paths=pseudo_gt_train_paths,
        gt_paths=gt_paths,
        radar_paths=radar_paths,
        image_path=image_path,
        data_path=data_path,
        patch_size=patch_size)
    val_dataset = BinaryClassificationDatasetVal(
        image_list=val_image_path,
        radar_points=val_radar_input_points,
        patch_size=patch_size)

# Set up data loader and data transforms
train_dataloader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers)

train_transforms = Transforms(
    normalized_image_range=normalized_image_range,
    crop_image_to_shape_on_point=patch_size,
    random_brightness=augmentation_random_brightness,
    random_noise_spread=augmentation_random_noise_spread)

val_dataloader = torch.utils.data.DataLoader(
    dataset=val_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=1,
    drop_last=False)

val_transforms = Transforms(
    normalized_image_range=normalized_image_range)

In [4]:
for batch_data in train_dataloader:
    # Load image (B x 3 x H x W), radar point (B x 3), and ground truth (B x 1 x H x W)
    image, radar_point, ground_truth = batch_data
    break


radar Path: ../data/radar_points-scene-id331/CAM_FRONT/n015-2018-09-27-15-33-17+0800__CAM_FRONT__1538033994762460-4.npy
Label Path: ../data/pseudo_ground_truth_train-scene-id331/CAM_FRONT/n015-2018-09-27-15-33-17+0800__CAM_FRONT__1538033994762460-248.png
Point IDX: 248
Image Path: /home/akash/Documents/nuScenes/samples/CAM_FRONT/n015-2018-09-27-15-33-17+0800__CAM_FRONT__1538033994762460.jpg
radar Path: ../data/radar_points-scene-id119/CAM_FRONT/n015-2018-08-01-17-13-57+0800__CAM_FRONT__1533115190362460-21.npy
Label Path: ../data/pseudo_ground_truth_train-scene-id119/CAM_FRONT/n015-2018-08-01-17-13-57+0800__CAM_FRONT__1533115190362460-313.png
Point IDX: 313
Image Path: /home/akash/Documents/nuScenes/samples/CAM_FRONT/n015-2018-08-01-17-13-57+0800__CAM_FRONT__1533115190362460.jpg
radar Path: ../data/radar_points-scene-id205/CAM_FRONT/n008-2018-08-06-15-06-32-0400__CAM_FRONT__1533582543162406-18.npy
Label Path: ../data/pseudo_ground_truth_train-scene-id205/CAM_FRONT/n008-2018-08-06-15-06-

In [5]:
radar_point

tensor([[ 977.0000,  516.0000,   38.1416],
        [1395.0000,  540.0000,   29.3133],
        [ 318.0000,  508.0000,   72.4119],
        [1135.0000,  510.0000,   50.3023],
        [ 394.0000,  489.0000,   35.5930],
        [ 858.0000,  505.0000,   73.3872],
        [ 442.0000,  580.0000,   22.7337],
        [ 191.0000,  517.0000,   39.8214],
        [ 685.0000,  469.0000,   77.0488],
        [1017.0000,  487.0000,   85.5349],
        [ 627.0000,  502.0000,   68.2461],
        [ 672.0000,  541.0000,   29.8461],
        [ 969.0000,  507.0000,   61.9448],
        [ 253.0000,  497.0000,   57.2263],
        [ 736.0000,  517.0000,   55.1877],
        [1304.0000,  534.0000,   19.6998],
        [1086.0000,  495.0000,   46.5097],
        [ 154.0000,  522.0000,   51.9376],
        [ 274.0000,  523.0000,   46.7974],
        [ 823.0000,  509.0000,   55.3949],
        [ 733.0000,  508.0000,   51.6581],
        [1280.0000,  560.0000,   20.9995],
        [1190.0000,  490.0000,   47.7974],
        [11

In [6]:
ground_truth.shape

torch.Size([64, 1, 900, 1600])

In [7]:
image.shape

torch.Size([64, 3, 900, 1600])

In [8]:
ground_truth[0,:,:,:].shape

torch.Size([1, 900, 1600])

In [9]:
np.where(ground_truth[10,:,:,:] == 1)

(array([0, 0]), array([325, 505]), array([633, 629]))